In [1]:
import pandas as pd

from sqlalchemy import create_engine

import numpy as np

from config import user, passw, host, port, name

In [3]:
physio = pd.read_csv('../data/physio.csv')
sleep = pd.read_csv('../data/sleeps.csv')
workouts = pd.read_csv('../data/workouts.csv')

In [6]:
physio.columns

Index(['Cycle start time', 'Cycle end time', 'Cycle timezone',
       'Recovery score %', 'Resting heart rate (bpm)',
       'Heart rate variability (ms)', 'Skin temp (celsius)', 'Blood oxygen %',
       'Day Strain', 'Energy burned (cal)', 'Max HR (bpm)', 'Average HR (bpm)',
       'Sleep onset', 'Wake onset', 'Sleep performance %',
       'Respiratory rate (rpm)', 'Asleep duration (min)',
       'In bed duration (min)', 'Light sleep duration (min)',
       'Deep (SWS) duration (min)', 'REM duration (min)',
       'Awake duration (min)', 'Sleep need (min)', 'Sleep debt (min)',
       'Sleep efficiency %'],
      dtype='object')

In [9]:
physio.rename(columns={
    "Cycle start time":"cycle_start",
    "Cycle end time":"cycle_end",
    "Cycle timezone":"cycle_timezone",
    "Recovery score %":"recovery_score",
    "Resting heart rate (bpm)":"resting_heart_rate",
    "Heart rate variability (ms)":"heart_rate_variability",
    "Skin temp (celsius)":"skin_temp",
    "Blood oxygen %":"blood_oxygen",
    "Day Strain":"day_strain",
    "Energy burned (cal)":"energy_burned",
    "Max HR (bpm)":"max_hr",
    "Average HR (bpm)":"average_hr",
    "Sleep onset":"sleep_onset",
    "Wake onset":"wake_onset",
    "Sleep performance %":"sleep_performance",
    "Respiratory rate (rpm)":"respiratory_rate",
    "Asleep duration (min)":"asleep_duration",
    "In bed duration (min)":"in_bed_duration",
    "Light sleep duration (min)":"light_sleep_duration",
    "Deep (SWS) duration (min)":"deep_sws_duration",
    "REM duration (min)":"rem_duration",
    "Awake duration (min)":"awake_duration",
    "Sleep need (min)":"sleep_need",
    "Sleep debt (min)":"sleep_debt",
    "Sleep efficiency %":"sleep_efficiency"
}, inplace=True)

physio.head()

,cycle_start,cycle_end,cycle_timezone,recovery_score,resting_heart_rate,heart_rate_variability,skin_temp,blood_oxygen,day_strain,energy_burned,...,respiratory_rate,asleep_duration,in_bed_duration,light_sleep_duration,deep_sws_duration,rem_duration,awake_duration,sleep_need,sleep_debt,sleep_efficiency
0,2023-02-21 22:59:07,NaN,UTC-05:00,58.0,51.0,54.0,35.20,96.54,NaN,NaN,...,15.5,366.0,406.0,245.0,74.0,47.0,40.0,469.0,0.0,90.0
1,2023-02-20 23:46:08,2023-02-21 22:59:07,UTC-05:00,89.0,49.0,62.0,34.60,97.00,2.9,1611.0,...,15.5,562.0,655.0,439.0,93.0,30.0,93.0,471.0,0.0,85.0
2,2023-02-19 23:09:10,2023-02-20 23:46:08,UTC-05:00,88.0,49.0,62.0,34.80,97.34,5.9,1720.0,...,15.4,575.0,684.0,368.0,118.0,89.0,109.0,487.0,12.0,85.0
3,2023-02-18 23:51:42,2023-02-19 23:09:10,UTC-05:00,72.0,52.0,55.0,34.80,97.17,8.4,1757.0,...,15.8,530.0,589.0,309.0,154.0,67.0,59.0,553.0,49.0,91.0
4,2023-02-17 22:33:06,2023-02-18 23:51:42,UTC-05:00,86.0,49.0,60.0,35.14,96.67,14.9,2640.0,...,16.2,374.0,445.0,272.0,68.0,34.0,71.0,471.0,0.0,86.0


In [10]:
sleep.columns

Index(['Cycle start time', 'Cycle end time', 'Cycle timezone', 'Sleep onset',
       'Wake onset', 'Sleep performance %', 'Respiratory rate (rpm)',
       'Asleep duration (min)', 'In bed duration (min)',
       'Light sleep duration (min)', 'Deep (SWS) duration (min)',
       'REM duration (min)', 'Awake duration (min)', 'Sleep need (min)',
       'Sleep debt (min)', 'Sleep efficiency %', 'Nap'],
      dtype='object')

In [11]:
sleep.rename(columns={
    'Cycle start time':'cycle_start', 
    'Cycle end time':'cycle_end', 
    'Cycle timezone':'cycle_timezone',
    'Sleep onset':'sleep_onset',
    'Wake onset':'wake_onset', 
    'Sleep performance %':'sleep_performance', 
    'Respiratory rate (rpm)':'repiratory_rate',
    'Asleep duration (min)':'asleep_duration',
    'In bed duration (min)':'in_bed_duration',
    'Light sleep duration (min)':'light_sleep_duration', 
    'Deep (SWS) duration (min)':'deep_sws_duration',
    'REM duration (min)':'rem_duration', 
    'Awake duration (min)':'awake_duration', 
    'Sleep need (min)':'sleep_need',
    'Sleep debt (min)':'sleep_debt', 
    'Sleep efficiency %':'sleep_efficiency', 
    'Nap':'nap'
}, inplace=True)

sleep.head()

,cycle_start,cycle_end,cycle_timezone,sleep_onset,wake_onset,sleep_performance,repiratory_rate,asleep_duration,in_bed_duration,light_sleep_duration,deep_sws_duration,rem_duration,awake_duration,sleep_need,sleep_debt,sleep_efficiency,nap
0,2023-02-21 22:59:07,NaN,UTC-05:00,2023-02-21 22:59:07,2023-02-22 05:39:30,78,15.5,366,406,245,74,47,40,469,0,90,False
1,2023-02-20 23:46:08,2023-02-21 22:59:07,UTC-05:00,2023-02-20 23:46:08,2023-02-21 10:41:45,100,15.5,562,655,439,93,30,93,471,0,85,False
2,2023-02-19 23:09:10,2023-02-20 23:46:08,UTC-05:00,2023-02-19 23:09:10,2023-02-20 10:34:05,100,15.4,575,684,368,118,89,109,487,12,85,False
3,2023-02-18 23:51:42,2023-02-19 23:09:10,UTC-05:00,2023-02-18 23:51:42,2023-02-19 09:11:11,96,15.8,530,589,309,154,67,59,553,49,91,False
4,2023-02-17 22:33:06,2023-02-18 23:51:42,UTC-05:00,2023-02-17 22:33:06,2023-02-18 05:58:34,79,16.2,374,445,272,68,34,71,471,0,86,False


In [12]:
workouts.columns

Index(['Cycle start time', 'Cycle end time', 'Cycle timezone',
       'Workout start time', 'Workout end time', 'Duration (min)',
       'Activity name', 'Activity Strain', 'Energy burned (cal)',
       'Max HR (bpm)', 'Average HR (bpm)', 'HR Zone 1 %', 'HR Zone 2 %',
       'HR Zone 3 %', 'HR Zone 4 %', 'HR Zone 5 %', 'GPS enabled',
       'Distance (meters)', 'Altitude gain (meters)',
       'Altitude change (meters)'],
      dtype='object')

In [13]:
workouts.rename(columns={
    'Cycle start time':'cycle_start',
    'Cycle end time':'cycle_end', 
    'Cycle timezone':'cycle_timezone',
    'Workout start time':'workout_start', 
    'Workout end time':'workout_end', 
    'Duration (min)':'duration',
    'Activity name':'activity_name', 
    'Activity Strain':'activity_strain', 
    'Energy burned (cal)':'calories_burned',
    'Max HR (bpm)':'max_hr', 
    'Average HR (bpm)':'average_hr', 
    'HR Zone 1 %':'hr_zone_1', 
    'HR Zone 2 %':'hr_zone_2',
    'HR Zone 3 %':'hr_zone_3', 
    'HR Zone 4 %':'hr_zone_4', 
    'HR Zone 5 %':'hr_zone_5', 
    'GPS enabled':'gps',
    'Distance (meters)':'distance',
    'Altitude gain (meters)':'altitude_gain',
    'Altitude change (meters)':'altitude_change'
}, inplace=True)

workouts.head()

,cycle_start,cycle_end,cycle_timezone,workout_start,workout_end,duration,activity_name,activity_strain,calories_burned,max_hr,average_hr,hr_zone_1,hr_zone_2,hr_zone_3,hr_zone_4,hr_zone_5,gps,distance,altitude_gain,altitude_change
0,2023-02-21 22:59:07,NaN,UTC-05:00,2023-02-22 09:59:28,2023-02-22 10:45:48,46,Weightlifting,4.9,93.0,116,93,34,63,2,0,0,False,NaN,NaN,NaN
1,2023-02-21 22:59:07,NaN,UTC-05:00,2023-02-22 07:34:34,2023-02-22 08:55:46,81,Tennis,11.9,588.0,148,123,6,15,27,50,0,False,NaN,NaN,NaN
2,2023-02-19 23:09:10,2023-02-20 23:46:08,UTC-05:00,2023-02-20 12:34:02,2023-02-20 14:45:06,131,Weightlifting,5.8,228.0,130,91,43,49,6,0,0,False,NaN,NaN,NaN
3,2023-02-18 23:51:42,2023-02-19 23:09:10,UTC-05:00,2023-02-19 11:52:24,2023-02-19 13:52:52,120,Weightlifting,7.4,314.0,137,100,16,65,17,0,0,False,NaN,NaN,NaN
4,2023-02-17 22:33:06,2023-02-18 23:51:42,UTC-05:00,2023-02-18 15:01:24,2023-02-18 16:40:04,98,Tennis,14.4,904.0,157,134,0,4,20,70,3,False,NaN,NaN,NaN


In [14]:
workouts.drop(columns=['altitude_gain', 'altitude_change'], inplace=True)

In [15]:
workouts.head()

,cycle_start,cycle_end,cycle_timezone,workout_start,workout_end,duration,activity_name,activity_strain,calories_burned,max_hr,average_hr,hr_zone_1,hr_zone_2,hr_zone_3,hr_zone_4,hr_zone_5,gps,distance
0,2023-02-21 22:59:07,NaN,UTC-05:00,2023-02-22 09:59:28,2023-02-22 10:45:48,46,Weightlifting,4.9,93.0,116,93,34,63,2,0,0,False,NaN
1,2023-02-21 22:59:07,NaN,UTC-05:00,2023-02-22 07:34:34,2023-02-22 08:55:46,81,Tennis,11.9,588.0,148,123,6,15,27,50,0,False,NaN
2,2023-02-19 23:09:10,2023-02-20 23:46:08,UTC-05:00,2023-02-20 12:34:02,2023-02-20 14:45:06,131,Weightlifting,5.8,228.0,130,91,43,49,6,0,0,False,NaN
3,2023-02-18 23:51:42,2023-02-19 23:09:10,UTC-05:00,2023-02-19 11:52:24,2023-02-19 13:52:52,120,Weightlifting,7.4,314.0,137,100,16,65,17,0,0,False,NaN
4,2023-02-17 22:33:06,2023-02-18 23:51:42,UTC-05:00,2023-02-18 15:01:24,2023-02-18 16:40:04,98,Tennis,14.4,904.0,157,134,0,4,20,70,3,False,NaN
